In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def lid_driven_cavity_fd(N=41, Re=100, max_iter=1000, tol=1e-6):
    """
    Solve lid-driven cavity using finite difference method with staggered grid
    
    Parameters:
    -----------
    N : int
        Number of grid points in each direction
    Re : float
        Reynolds number
    max_iter : int
        Maximum number of iterations
    tol : float
        Convergence tolerance
    """
    # Domain setup
    L = 1.0  # Length of the cavity
    U = 1.0  # Lid velocity
    nu = U * L / Re  # Kinematic viscosity
    dx = L / (N-1)
    dy = dx
    dt = 0.1 * dx  # Time step (CFL condition)
    
    # Grid setup (staggered grid)
    x = np.linspace(0, L, N)
    y = np.linspace(0, L, N)
    X, Y = np.meshgrid(x, y)
    
    # Initialize variables
    u = np.zeros((N, N))     # x-velocity
    v = np.zeros((N, N))     # y-velocity
    p = np.zeros((N, N))     # pressure
    b = np.zeros((N, N))     # divergence
    
    # Boundary condition for lid
    u[-1, :] = U  # Top wall moving with velocity U
    
    # Main loop
    for it in range(max_iter):
        un = u.copy()
        vn = v.copy()
        
        # Compute intermediate velocities
        for i in range(1, N-1):
            for j in range(1, N-1):
                # x-momentum
                u[i,j] = un[i,j] + dt * (
                    - 0.25*((un[i+1,j] + un[i,j])**2 - (un[i,j] + un[i-1,j])**2)/dx
                    - 0.25*((un[i,j+1] + un[i,j])*(vn[i,j+1] + vn[i-1,j+1]) - 
                           (un[i,j] + un[i,j-1])*(vn[i,j] + vn[i-1,j]))/dy
                    + nu*(un[i+1,j] - 2*un[i,j] + un[i-1,j])/dx**2
                    + nu*(un[i,j+1] - 2*un[i,j] + un[i,j-1])/dy**2
                    - (p[i,j+1] - p[i,j])/dx
                )
                
                # y-momentum
                v[i,j] = vn[i,j] + dt * (
                    - 0.25*((un[i+1,j] + un[i+1,j-1])*(vn[i+1,j] + vn[i,j]) - 
                           (un[i,j] + un[i,j-1])*(vn[i,j] + vn[i-1,j]))/dx
                    - 0.25*((vn[i,j+1] + vn[i,j])**2 - (vn[i,j] + vn[i,j-1])**2)/dy
                    + nu*(vn[i+1,j] - 2*vn[i,j] + vn[i-1,j])/dx**2
                    + nu*(vn[i,j+1] - 2*vn[i,j] + vn[i,j-1])/dy**2
                    - (p[i+1,j] - p[i,j])/dy
                )
        
        # Enforce boundary conditions
        u[0, :] = 0  # Bottom wall
        u[:, 0] = 0  # Left wall
        u[:, -1] = 0  # Right wall
        u[-1, :] = U  # Top wall (lid)
        
        v[0, :] = 0  # Bottom wall
        v[-1, :] = 0  # Top wall
        v[:, 0] = 0  # Left wall
        v[:, -1] = 0  # Right wall
        
        # Solve pressure Poisson equation
        for _ in range(50):  # Gauss-Seidel iterations for pressure
            pn = p.copy()
            for i in range(1, N-1):
                for j in range(1, N-1):
                    b[i,j] = (u[i,j+1] - u[i,j])/dx + (v[i+1,j] - v[i,j])/dy
                    p[i,j] = 0.25 * (
                        p[i+1,j] + p[i-1,j] + p[i,j+1] + p[i,j-1]
                        - dx*dy*b[i,j]/dt
                    )
            
            # Pressure boundary conditions (Neumann)
            p[0, :] = p[1, :]
            p[-1, :] = p[-2, :]
            p[:, 0] = p[:, 1]
            p[:, -1] = p[:, -2]
        
        # Check convergence
        error = np.max(np.abs(u - un)) + np.max(np.abs(v - vn))
        if error < tol:
            print(f"Converged after {it} iterations")
            break
            
    return u, v, p, X, Y

# Solve and visualize
N = 41
Re = 100
u, v, p, X, Y = lid_driven_cavity_fd(N, Re)

# Plot results
plt.figure(figsize=(15, 5))

# Streamlines
plt.subplot(131)
speed = np.sqrt(u**2 + v**2)
plt.streamplot(X, Y, u, v, density=2, color='blue')
plt.title('Streamlines')
plt.xlabel('x')
plt.ylabel('y')

# Velocity magnitude
plt.subplot(132)
plt.contourf(X, Y, speed, levels=50, cmap='viridis')
plt.colorbar(label='Velocity magnitude')
plt.title('Velocity Magnitude')
plt.xlabel('x')
plt.ylabel('y')

# Pressure contours
plt.subplot(133)
plt.contourf(X, Y, p, levels=50, cmap='RdBu')
plt.colorbar(label='Pressure')
plt.title('Pressure Field')
plt.xlabel('x')
plt.ylabel('y')

plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'scipy'